In [1]:
import os
import glob
import pandas as pd

import numpy as np
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.transform import jitter
import seaborn as sns
import matplotlib
from bokeh.models import HoverTool
from scipy import stats
import bokeh_catplot
import pandas as pd
import math

from bokeh.layouts import row
bokeh.io.output_notebook()

from tkinter import Tk
from tkinter.filedialog import askdirectory

Loading BokehJS ...

In [2]:
#Functions
def upgradeCSV(data_dir, app_root_dir, save_folder, filename_distinct_substring, TechRep):
    extension = 'csv'
    list_csv = []
    for root, dirs, files in os.walk(data_dir, topdown = False):
        for name in files:
            if extension in name:
                if filename_distinct_substring in name:
                    list_csv.append(os.path.join(root, name))
    os.chdir(app_root_dir)
    app_data_dir = "Data\\"
    if not os.path.exists(app_data_dir):
        os.mkdir(app_data_dir)
    os.chdir(app_data_dir)
    new_files_dir = save_folder
    if not os.path.exists(new_files_dir):
        os.mkdir(new_files_dir)
    os.chdir(new_files_dir)
    for file in list_csv:
        df_temp = pd.read_csv(file, sep = ',', na_values = '*')
        names = file.rsplit('\\',1)[1].rsplit('_',4)
        df_temp['Strain'] = names[0]
        df_temp['Reporter'] = names[1]
        df_temp['TimePoint'] = names[2].rsplit('h',1)[0]
        df_temp['TeckRep'] = TechRep
        df_temp.to_csv(file.rsplit('\\',1)[1], index = None, header = True)
        
def msd(xdata, ydata):
    r=np.sqrt(xdata**2+ydata**2)
    diff=np.diff(r)
    diff_sq=diff**2
    MSD=np.mean(diff_sq)
    return MSD

def rmsd(xdata, ydata):
    xdata_x0=np.subtract(xdata, xdata[0])
    x=xdata_x0[1:]
    ydata_y0=np.subtract(ydata, ydata[0])
    y=ydata_y0[1:]
    RMSD=np.sqrt(np.mean(x**2+y**2))
    return RMSD

def bootstrap_sampling(my_array, bootstrap_samples, bootstrap_replicates):
    bs_samples=np.zeros((bootstrap_replicates, bootstrap_samples))
    bs_rep=np.arange(bootstrap_replicates)
    for i in bs_rep:
        bs_samples[i, :]=np.random.choice(my_array, bootstrap_samples)
    return bs_samples

def bootstrap_stats(bs_samples):
    samples_shape=np.shape(bs_samples)
    bs_means=np.zeros((samples_shape[0], 1))
    bs_IC_means=np.zeros((1, 2))
    bs_medians=np.zeros((samples_shape[0], 1))
    bs_IC_medians=np.zeros((1, 2))
    for i in np.arange(samples_shape[0]):
        bs_means[i]=np.mean(bs_samples[i, :])
        bs_medians[i]=np.median(bs_samples[i, :])
    bs_IC_means[0, 0]=np.quantile(bs_means, .025)
    bs_IC_means[0, 1]=np.quantile(bs_means, .975)
    bs_IC_medians[0, 0]=np.quantile(bs_medians, .025)
    bs_IC_medians[0, 1]=np.quantile(bs_medians, .975)
    return np.mean(bs_means), np.median(bs_medians), bs_IC_means, bs_IC_medians

In [4]:
#set root folder
data_dir = "D:\\Motility\\"
app_root_dir = "C:\\Users\\tala\\Desktop\\Python-Code\\AnalysisMotility\\"
save_folder = "upgraded_csv_files\\"
upgradeCSV(data_dir, app_root_dir, save_folder, "track", 1)
upgradeCSV(data_dir, app_root_dir, save_folder, "spot", 1)
upgradeCSV(data_dir, app_root_dir, save_folder, "fluo", 1)

os.chdir(app_root_dir + "Data\\" + save_folder)
extension = 'csv'
list_track_csv = [i for i in glob.glob('*track*.{}'.format(extension))]
list_spot_csv = [i for i in glob.glob('*spot*.{}'.format(extension))]
list_fluo_csv = [i for i in glob.glob('*fluo*.{}'.format(extension))]

combined_track_csv = pd.concat([pd.read_csv(f) for f in list_track_csv ], sort=False)
combined_spot_csv = pd.concat([pd.read_csv(f) for f in list_spot_csv ], sort=False)
combined_fluo_csv = pd.concat([pd.read_csv(f) for f in list_fluo_csv ], sort=False)
os.chdir(app_root_dir + "Data\\")
combined_track_csv.to_csv( "TrackData.csv", index=False, encoding='utf-8-sig')
combined_spot_csv.to_csv( "SpotData.csv", index=False, encoding='utf-8-sig')
combined_fluo_csv.to_csv( "FluoData.csv", index=False, encoding='utf-8-sig')

In [5]:
df_spots = combined_spot_csv.copy()
df_tracks = combined_track_csv.copy()
df_fluo = combined_fluo_csv.copy()
df_tracks['MSD']=0.0
df_tracks['RMSD']=0.0
df_tracks['PaqaTotalFluorescence']='nan'
df_tracks['rfpTotalFluorescence']='nan'
df_tracks['PaQa_RFP_ratio']='nan'
df_fluo.head()

,,Label,Track_ID,BiologicalReplicate,PaqaArea,PaqaMean,PaqaTotalFluorescence,PaqaMin,PaqaMax,PaqaStd,...,rfpMean,rfpTotalFluorescence,rfpMin,rfpMax,rfpStd,PaQa_RFP_ratio,Strain,Reporter,TimePoint,TeckRep
0,1,fliC-_PaQa_0h001.nd2 - C=2.tif,0,1,2.85188,58.81481,167.73251,12,127,32.22163,...,129.91304,315.60752,52,285,71.40213,0.53146,fliC-,PaQa,0,1
1,2,fliC-_PaQa_0h001.nd2 - C=2.tif,1,1,2.53500,76.25000,193.29376,15,149,38.95622,...,169.34783,411.40940,59,373,96.80327,0.46983,fliC-,PaQa,0,1
2,3,fliC-_PaQa_0h001.nd2 - C=2.tif,2,1,2.42938,72.00000,174.91501,27,160,34.18931,...,154.28571,342.22502,74,278,63.97667,0.51111,fliC-,PaQa,0,1
3,4,fliC-_PaQa_0h001.nd2 - C=2.tif,4,1,2.95750,245.64286,726.48879,103,486,118.68140,...,201.70370,575.23378,90,421,85.26917,1.26295,fliC-,PaQa,0,1
4,5,fliC-_PaQa_0h001.nd2 - C=2.tif,5,1,2.21813,61.52381,136.46751,24,130,31.99784,...,143.35000,302.82689,44,344,91.42801,0.45065,fliC-,PaQa,0,1


In [6]:
Strains=list(df_spots.Strain.unique())
TimePoints=list(df_spots.TimePoint.unique())
print(Strains)
print(TimePoints)

['fliC-', 'pilG-fliC-', 'pilH-fliC-', 'pilT-pilA-fliC-']
[0, 1, 2, 3]


In [7]:
n=0
boot_strain=list()
boot_tp=np.zeros((len(Strains)*len(TimePoints), 1))
print(np.shape(boot_tp))
boot_meanMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_meanRMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_medianRMSD=np.zeros((len(Strains)*len(TimePoints), 1))
boot_IC_meanMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_medianMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_meanRMSD=np.zeros((len(Strains)*len(TimePoints), 2))
boot_IC_medianRMSD=np.zeros((len(Strains)*len(TimePoints), 2))
for s, strain in enumerate(Strains):
    print(strain)
    for t, tp in enumerate(TimePoints):
        track_IDs=list(df_spots.TRACK_ID.loc[(df_spots['Strain']==strain) & (df_spots['TimePoint']==tp)].unique())
        track_Label=list(df_tracks.Label.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].unique())
        print('In Time point '+ str(tp)+' there are '+str(len(track_IDs))+' tracks in SpotData.csv and '+str(len(track_Label))+' in TrackData.csv')
        for track in track_IDs:
            xdata=df_spots.POSITION_X.loc[(df_spots['Strain']==strain) & (df_spots['TimePoint']==tp) & (df_spots['TRACK_ID']==track)].values
            ydata=df_spots.POSITION_Y.loc[(df_spots['Strain']==strain) & (df_spots['TimePoint']==tp) & (df_spots['TRACK_ID']==track)].values
            paqa=df_fluo.PaqaTotalFluorescence.loc[(df_fluo['Strain']==strain) & (df_fluo['TimePoint']==tp) & (df_fluo['Track_ID']==track)].values
            rfp=df_fluo.rfpTotalFluorescence.loc[(df_fluo['Strain']==strain) & (df_fluo['TimePoint']==tp) & (df_fluo['Track_ID']==track)].values
            ratio=df_fluo.PaQa_RFP_ratio.loc[(df_fluo['Strain']==strain) & (df_fluo['TimePoint']==tp) & (df_fluo['Track_ID']==track)].values
            MSD=msd(xdata, ydata)
            RMSD=rmsd(xdata, ydata)
            df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'MSD']=MSD
            df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'RMSD']=RMSD
            if bool(paqa):
                df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'PaqaTotalFluorescence']=paqa
                df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'rfpTotalFluorescence']=rfp
                df_tracks.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp) & (df_tracks['Label']=='Track_'+str(track)), 'PaQa_RFP_ratio']=ratio
        boot_strain.append(strain)
        boot_tp[n]=tp
        tracks_data_MSD=df_tracks.MSD.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].values
        bs_tracks_data_MSD=bootstrap_sampling(tracks_data_MSD, len(tracks_data_MSD), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_tracks_data_MSD)
        boot_meanMSD[n]=bs_means
        boot_IC_meanMSD[n,:]=IC_means
        boot_medianMSD[n]=bs_medians
        boot_IC_medianMSD[n,:]=IC_medians
        tracks_data_RMSD=df_tracks.RMSD.loc[(df_tracks['Strain']==strain) & (df_tracks['TimePoint']==tp)].values
        bs_tracks_data_RMSD=bootstrap_sampling(tracks_data_RMSD, len(tracks_data_RMSD), 1000)
        [bs_means, bs_medians, IC_means, IC_medians]=bootstrap_stats(bs_tracks_data_RMSD)
        boot_meanRMSD[n]=bs_means
        boot_IC_meanRMSD[n,:]=IC_means
        boot_medianRMSD[n]=bs_medians
        boot_IC_medianRMSD[n,:]=IC_medians
        n=n+1
n=0

(16, 1)
fliC-
In Time point 0 there are 96 tracks in SpotData.csv and 96 in TrackData.csv
In Time point 1 there are 19 tracks in SpotData.csv and 19 in TrackData.csv
In Time point 2 there are 20 tracks in SpotData.csv and 20 in TrackData.csv


C:\Users\tala\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


In Time point 3 there are 27 tracks in SpotData.csv and 27 in TrackData.csv
pilG-fliC-
In Time point 0 there are 98 tracks in SpotData.csv and 98 in TrackData.csv
In Time point 1 there are 29 tracks in SpotData.csv and 29 in TrackData.csv
In Time point 2 there are 34 tracks in SpotData.csv and 34 in TrackData.csv
In Time point 3 there are 34 tracks in SpotData.csv and 34 in TrackData.csv
pilH-fliC-
In Time point 0 there are 9 tracks in SpotData.csv and 9 in TrackData.csv
In Time point 1 there are 16 tracks in SpotData.csv and 16 in TrackData.csv
In Time point 2 there are 14 tracks in SpotData.csv and 14 in TrackData.csv
In Time point 3 there are 5 tracks in SpotData.csv and 5 in TrackData.csv
pilT-pilA-fliC-
In Time point 0 there are 85 tracks in SpotData.csv and 85 in TrackData.csv
In Time point 1 there are 17 tracks in SpotData.csv and 17 in TrackData.csv
In Time point 2 there are 14 tracks in SpotData.csv and 14 in TrackData.csv
In Time point 3 there are 12 tracks in SpotData.csv an

In [8]:
names = ['TimePoint', 'meanMSD','medianMSD', 'meanRMSD', 'medianRMSD']
data = np.concatenate((boot_tp, boot_meanMSD, boot_medianMSD, boot_meanRMSD, boot_medianRMSD), axis=1)
df_boot_stats=pd.DataFrame(data=np.transpose(data), index=names).T
df_boot_stats['Strain']=boot_strain
df_boot_stats['IC_meanMSD']=list(boot_IC_meanMSD)
df_boot_stats['IC_medianMSD']=list(boot_IC_medianMSD)
df_boot_stats['IC_meanRMSD']=list(boot_IC_meanRMSD)
df_boot_stats['IC_medianRMSD']=list(boot_IC_medianRMSD)
df_boot_stats['TeckRep']=1
df_boot_stats.head()

,TimePoint,meanMSD,medianMSD,meanRMSD,medianRMSD,Strain,IC_meanMSD,IC_medianMSD,IC_meanRMSD,IC_medianRMSD,TeckRep
0,0.0,0.117082,0.003187,0.444389,0.289718,fliC-,"[0.026594596497385155, 0.2318448495018382]","[0.0027263841672385825, 0.003706423529356574]","[0.38958543499625675, 0.5073584855844404]","[0.2678936279954961, 0.33094924713918783]",1
1,1.0,0.328406,0.011352,2.041360,1.712932,fliC-,"[0.037135799293829284, 0.7675534380273048]","[0.005815094671687431, 0.03680311245756276]","[1.652535898461977, 2.4536336755796504]","[1.4673472798668667, 2.3214322230153757]",1
2,2.0,0.332515,0.145627,3.877224,2.839030,fliC-,"[0.18963069226200296, 0.5001019903525734]","[0.08161096006004787, 0.23685856320077095]","[2.808362952463112, 5.189505108063684]","[1.953730938903373, 4.486271768268913]",1
3,3.0,3.463440,0.286529,3.403745,2.267266,fliC-,"[1.8686973157376354, 5.4084538477841235]","[0.12686692349408907, 1.3440317751365027]","[2.7595057445956854, 4.124090502886831]","[1.7260133098354673, 3.9641643026012803]",1
4,0.0,0.052044,0.013533,0.634048,0.459455,pilG-fliC-,"[0.04246994833631106, 0.06244120362181843]","[0.008192046910709072, 0.030824975011654973]","[0.5655527073520218, 0.6997461100711981]","[0.3716910294813926, 0.5432660067618849]",1


In [9]:
df_tracks.to_csv( "TrackDataMSD.csv", index=False, encoding='utf-8-sig')

In [10]:
means = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].mean().to_frame().reset_index()
stds = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].std().to_frame().reset_index()
medians = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].median().to_frame().reset_index()
sems = df_tracks.groupby(['Strain', 'TimePoint', 'TeckRep'])['RMSD'].sem().to_frame().reset_index()

In [11]:
colors = bokeh.palettes.d3['Category10'][10]
parameter='RMSD'

p = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='log',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter + ' (µm)',
    title="Root mean squared displacement (median, sem)"
)

p1 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='log',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter + ' (µm)',
    title="Root mean squared displacement (mean, sem)"
)

In [12]:

for s, strain in enumerate(Strains):
    p.line(
        source = medians.loc[(medians['Strain'] == strain),['Strain','TimePoint',parameter]],
        x = 'TimePoint',
        y = parameter,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p.circle(
        source = medians.loc[(medians['Strain'] == strain),['Strain','TimePoint',parameter]],
        x = 'TimePoint',
        y = parameter,
        color = colors[s],
        alpha=0.8,
        size=5
    )

for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=float(medians[(medians['Strain'] == strain ) & (medians['TimePoint'] == tp)][parameter])
            sem=float(sems[(sems['Strain'] == strain ) & (sems['TimePoint'] == tp)][parameter])
            if (m-sem < 0):
                a = [0, m+sem]
            else: a = [m-sem, m+sem]
            p.line(
                x = b,
                y = a,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )

for s, strain in enumerate(Strains):
    p1.line(
        source = means.loc[(medians['Strain'] == strain),['Strain','TimePoint',parameter]],
        x = 'TimePoint',
        y = parameter,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p1.circle(
        source = means.loc[(medians['Strain'] == strain),['Strain','TimePoint',parameter]],
        x = 'TimePoint',
        y = parameter,
        color = colors[s],
        alpha=0.8,
        size=5
    )  

for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=float(means[(means['Strain'] == strain ) & (means['TimePoint'] == tp)][parameter])
            sem=float(sems[(sems['Strain'] == strain ) & (sems['TimePoint'] == tp)][parameter])
            if (m-sem < 0):
                a = [0, m+sem]
            else: a = [m-sem, m+sem]
            
            p1.line(
                x = b,
                y = a,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )

p.output_backend = 'svg'
p1.output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(p, p1))

In [13]:
colors = bokeh.palettes.d3['Category10'][10]
parameter1='meanRMSD'
parameter2='medianRMSD'
IC1='IC_meanRMSD'
IC2='IC_medianRMSD'
p2 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter1 + ' (µm)',
    title="Root mean squared displacement (bootstrap mean, 95% IC)"
)

p3 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='linear',
    x_axis_label = 'Time (h)',
    y_axis_label = parameter2 + ' (µm)',
    title="Root mean squared displacement (bootstrap median, 95% IC)"
)

In [14]:

for s, strain in enumerate(Strains):
    p2.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p2.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter1]],
        x = 'TimePoint',
        y = parameter1,
        color = colors[s],
        alpha=0.8,
        size=5
    )


for s, strain in enumerate(Strains):
    p3.line(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter2]],
        x = 'TimePoint',
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        legend = 'Strain'
    )
    p3.circle(
        source = df_boot_stats.loc[(df_boot_stats['Strain'] == strain), ['Strain', 'TimePoint', parameter2]],
        x = 'TimePoint',
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        size=5
    )  

for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter1]].values[0][0]
            ic=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [IC1]].values[0][0]
            if (m-ic[0] < 0):
                a = [0, m+ic[1]]
            else: a = [m-ic[0], m+ic[1]]           
            p2.line(
                x = b,
                y = ic,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )
for s, strain in enumerate(Strains):
        for tp in TimePoints:
            b = [tp,tp]
            m=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [parameter2]].values[0][0]
            ic=df_boot_stats.loc[(df_boot_stats['Strain'] == strain) & (df_boot_stats['TimePoint'] == tp), [IC2]].values[0][0]
            if (m-ic[0] < 0):
                a = [0, m+ic[1]]
            else: a = [m-ic[0], m+ic[1]]            
            p3.line(
                x = b,
                y = ic,
                color = colors[s],
                alpha=0.5,
                line_width=3
            )

p2.output_backend = 'svg'
p3.output_backend = 'svg'

bokeh.io.show(bokeh.layouts.row(p2, p3))

In [16]:
colors = bokeh.palettes.d3['Category10'][10]
parameter1='RMSD'
parameter2='PaQa_RFP_ratio'
p5 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='log',
    y_axis_type='log',
    x_axis_label = parameter1,
    y_axis_label = parameter2,
    title="PaQa/RFP ratio vs RMSD"
)
Strains2=['fliC-', 'pilH-fliC-']
for s, strain in enumerate(Strains):
    p5.circle(
        source = df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter1, parameter2]],
        x = parameter1,
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        size=5,
        legend=strain
    )

bokeh.io.show(p5)

In [17]:
for strain in Strains:
    X=np.array(df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter1]].values, dtype=np.float32)
    Y=np.array(df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter2]].values, dtype=np.float32)
    print(parameter1+' vs '+parameter2+' in '+strain+':')
    [spearman_r, spearman_p]=stats.spearmanr(X, Y)
    print('Spearman correlation = '+str(spearman_r)+', p-value = '+ str(spearman_p))
    [pearson_r, pearson_p]=stats.pearsonr(X, Y)
    print('Pearson correlation = '+str(pearson_r)+', p-value = '+str(pearson_p))
    print('')

RMSD vs PaQa_RFP_ratio in fliC-:
Spearman correlation = 0.2710770303241548, p-value = 1.7434018741655457e-06
Pearson correlation = [0.29746097], p-value = [1.3822127e-07]

RMSD vs PaQa_RFP_ratio in pilG-fliC-:
Spearman correlation = -0.09093689815339301, p-value = 0.07284291698250485
Pearson correlation = [-0.03789327], p-value = [0.45555153]

RMSD vs PaQa_RFP_ratio in pilH-fliC-:
Spearman correlation = 0.25708502024291496, p-value = 0.023074937828352373
Pearson correlation = [0.30766037], p-value = [0.00614177]

RMSD vs PaQa_RFP_ratio in pilT-pilA-fliC-:
Spearman correlation = 0.0019334887008446483, p-value = 0.9763291240230205
Pearson correlation = [0.05289453], p-value = [0.41662455]



In [18]:
colors = bokeh.palettes.d3['Category10'][10]
parameter1='TimePoint'
parameter2='PaQa_RFP_ratio'
p6 = bokeh.plotting.figure(
    width=600, 
    height=600,  
    x_axis_type='linear',
    y_axis_type='log',
    x_axis_label = parameter1,
    y_axis_label = parameter2,
    title="PaQa/RFP ratio vs timePoint"
)
Strains2=['fliC-', 'pilH-fliC-']
for s, strain in enumerate(Strains2):
    p6.circle(
        source = df_tracks.loc[(df_tracks['Strain'] == strain) & (df_tracks['TRACK_START'] == 0), [parameter1, parameter2]],
        x = parameter1,
        y = parameter2,
        color = colors[s],
        alpha=0.8,
        size=5,
        legend=strain
    )

bokeh.io.show(p6)